**Important:** The model estimation code is intended to work with an experimental parallelised Vensim engine. With appropriate modifications to the main function calls (but not the analytical procedure), the same analysis can be run on regular commercially available Vensim DSS, though it will take *much* longer. Please contact [Tom Fiddaman](mailto:tom@ventanasystems.com) for information on the experimental Vensim engine.

For more information on the model estimation procedure, see S1 of the Supplementary Materials of the paper.

In [ ]:
import os
import subprocess
import regex
import json
import time
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import statsmodels.api as sm

from shutil import copy
from scipy.stats import pearsonr
from distutils.dir_util import copy_tree

from statsmodels.stats.outliers_influence import variance_inflation_factor

##### CLASS & FUNCTION DEFINITIONS FOR WORKING WITH VENSIM/VENGINE #####

class Script(object):
    """Master object for holding and modifying .cmd script settings, 
    creating .cmd files, and running them through Vensim/Vengine"""
    def __init__(self, controlfile):
        print("Initialising", self)
        for k, v in controlfile['simsettings'].items():
            self.__setattr__(k, v if isinstance(v, str) else v.copy())
        self.setvals = []
        self.runcmd = "MENU>RUN_OPTIMIZE|o\n"
        self.savecmd = f"MENU>VDF2TAB|!|!|{self.savelist}|\n"
        self.basename = controlfile['baserunname']
        self.cmdtext = []
        
    def copy_model_files(self, dirname):
        """Create subdirectory and copy relevant model files to it,
        then change working directory to subdirectory"""
        os.makedirs(dirname, exist_ok=True)
        os.chdir(f"./{dirname}")

        # Copy needed files from the working directory into the sub-directory
        for s in ['model', 'payoff', 'optparm', 'sensitivity', 'savelist', 'senssavelist']:
            if getattr(self, s):
                copy(f"../{getattr(self, s)}", "./")
        for slist in ['data', 'changes']:
            for file in getattr(self, slist):
                copy(f"../{file}", "./")
            
    def add_suffixes(self, settingsfxs):
        """Cleanly modifies .cmd script settings with specified suffixes"""
        for s, sfx in settingsfxs.items():
            if hasattr(self, s):
                self.__setattr__(s, getattr(self, s)[:-4] + sfx + getattr(self, s)[-4:])
   
    def update_changes(self, chglist, setvals=[]):
        """Reformats chglist as needed to extend changes settings; 
        see compile_script for details"""
        # Combines and flattens list of paired change names & suffixes
        flatlist = [i for s in 
                    [[f"{self.basename}_{n}_{sfx}.out" for n in name] 
                     if isinstance(name, list) else [f"{self.basename}_{name}_{sfx}.out"] 
                     for name, sfx in chglist] for i in s]
        self.changes.extend(flatlist)
        self.setvals = setvals
          
    def write_script(self, scriptname):
        """Compiles and writes actual .cmd script file"""
        self.cmdtext.extend(["SPECIAL>NOINTERACTION\n", 
                             f"SPECIAL>LOADMODEL|{self.model}\n"])
        
        for s in ['payoff', 'sensitivity', 'optparm', 'savelist', 'senssavelist']:
            if hasattr(self, s):
                self.cmdtext.append(f"SIMULATE>{s}|{getattr(self, s)}\n")
        
        if hasattr(self, 'data'):
            datatext = ','.join(self.data)
            self.cmdtext.append(f"SIMULATE>DATA|\"{','.join(self.data)}\"\n")

        if self.changes:
            self.cmdtext.append(f"SIMULATE>READCIN|{self.changes[0]}\n")
            for file in self.changes[1:]:
                self.cmdtext.append(f"SIMULATE>ADDCIN|{file}\n")
        
        self.cmdtext.extend(["\n", f"SIMULATE>RUNNAME|{scriptname}\n", 
                             self.runcmd, self.savecmd, 
                             "SPECIAL>CLEARRUNS\n", "MENU>EXIT\n"])
        
        with open(f"{scriptname}.cmd", 'w') as scriptfile:
            scriptfile.writelines(self.cmdtext)
    
    def run_script(self, scriptname, controlfile, subdir, logfile):
        """Runs .cmd script file using function robust to 
        Vengine errors, and returns payoff value if applicable"""
        return run_vengine_script(scriptname, controlfile['vensimpath'], 
                                  controlfile['timelimit'], '.log', check_opt, logfile)

    
class CtyScript(Script):
    """Script subclass for country optimization runs"""
    def __init__(self, controlfile):
        super().__init__(controlfile)
        self.genparams = controlfile['analysissettings']['genparams'].copy()
        
    def prep_subdir(self, scriptname, controlfile, subdir):
        """Creates subdirectory for country-specific files and output"""
        self.copy_model_files(subdir)
        copy(f"../{scriptname}.cmd", "./")
        self.genparams.append(f"[{subdir}]")
        for file in self.changes:
            if 'main' in file:
                clean_outfile(file, self.genparams)
            
    def run_script(self, scriptname, controlfile, subdir, logfile):
        self.prep_subdir(scriptname, controlfile, subdir)
        res = run_vengine_script(scriptname, controlfile['vensimpath'], 
                                 controlfile['timelimit'], '.log', check_opt, logfile)
        copy(f"./{scriptname}.out", "..") # Copy the .out file to parent directory
        os.chdir("..")
        return res


def compile_script(controlfile, scriptclass, name, namesfx, settingsfxs, 
                   logfile, chglist=[], setvals=[], subdir=None):
    """Master function for assembling & running .cmd script
    
    Parameters
    ----------
    controlfile : JSON object
        Master control file specifying sim settings, runname, etc.
    scriptclass : Script object
        Type of script object to instantiate, depending on run type
    name : str
    namesfx : str
        Along with `name`, specifies name added to baserunname for run
    settingsfxs : dict of str
        Dict of suffixes to append to filenames in simsettings; use to 
        distinguish versions of e.g. .mdl, .voc, .vpd etc. files
    logfile : str of filename/path
    chglist : list of tuples of (str or list, str)
        Specifies changes files to be used in script; specify as tuples 
        corresponding to `name`, `namesfx` of previous run .out to use; 
        tuples can also take a list of `names` as first element, taking 
        each with the same second element; if used with ScenScript run, 
        `chglist` can also take one non-tuple str as its last element, 
        which will be added directly (e.g. .cin files for scenarios)
    setvals : list of tuples of (str, int or float, <str>)
        Specifies variables and values to change for a given run using 
        Vensim's SETVAL script command; by default all SETVAL commands 
        will be implemented together for main run, but if `scriptclass` 
        is MultiScript, each SETVAL command will be implemented and run 
        separately in sequence; if used with MultiScript, each tuple in 
        `setvals` will require a third str element specifying the suffix 
        with which to save the run
    subdir : str, optional
        Name of subdirectory to create/use for run, if applicable
    
    Returns
    -------
    float
        Payoff value of the script run, if applicable, else 0
    """
    mainscript = scriptclass(controlfile)
    mainscript.add_suffixes(settingsfxs)
    mainscript.update_changes(chglist, setvals)
    scriptname = f"{mainscript.basename}_{name}_{namesfx}"    
    mainscript.write_script(scriptname)
    return mainscript.run_script(scriptname, controlfile, subdir, logfile)


def check_opt(scriptname, logfile):
    """Check function for use with run_vengine_script for optimizations"""
    if check_zeroes(scriptname):
        write_log(f"Help! {scriptname} is being repressed!", logfile)
    return not check_zeroes(scriptname)


def run_vengine_script(scriptname, vensimpath, timelimit, checkfile, check_func, logfile):
    cmd_file_path = f"./{scriptname}.cmd"
    print("Command file path: ", cmd_file_path)
        
    if os.path.exists(cmd_file_path) and os.path.exists(vensimpath):
        write_log(f"Running {scriptname}", logfile)
        subprocess.run([vensimpath, cmd_file_path])
    else:
        write_log("Error: Either the command file or the Vensim executable path does not exist.", logfile)


def clean_outfile(outfilename, linekey):
    """Clean an outfile to include only lines containing a string in 
    `linekey`, which should be a list of strings to keep"""
    with open(outfilename,'r') as f:
        filedata = f.readlines()

    newdata = [line for line in filedata if any(k in line for k in linekey)]
    
    with open(outfilename, 'w') as f:
        f.writelines(newdata)


def check_zeroes(scriptname):
    """Check if an .out file has any parameters set to zero (indicates 
    Vengine error), return True if any parameters zeroed OR if # runs = 
    # restarts, and False otherwise"""
    filename = f"{scriptname}.out"
    with open(filename,'r') as f0:
        filedata = f0.readlines()
    
    checklist = []
    for line in filedata:
        if line[0] != ':':
            if ' = 0 ' in line:
                checklist.append(True)
            else:
                checklist.append(False)
        elif ':RESTART_MAX' in line:
            restarts = regex.findall(r'\d+', line)[0]
    
    # Ensure number of simulations != number of restarts
    if f"After {restarts} simulations" in filedata[0]:
        checklist.append(True)
        
    # Ensure payoff is not erroneous
    if abs(read_payoff(filename)) == 1.29807e+33:
        checklist.append(True)
    
    return any(checklist)


def write_log(string, logfile):
    """Writes printed script output to a logfile"""
    with open(logfile,'a') as f:
        f.write(string + "\n")
    print(string)

    
def modify_mdl(country, finaltime, modelname, newmodelname):
    """Opens .mdl as text, identifies Rgn subscript, and replaces 
    with appropriate country name"""
    with open(modelname,'r') as f:
        filedata = f.read()
        
    rgnregex = regex.compile(r"Rgn(\s)*?:(\n)?[\s\S]*?(\n\t~)")
    timeregex = regex.compile(r"FINAL TIME\s*=\s*\d*\n")
    tempdata = rgnregex.sub(f"Rgn:\n\t{country}\n\t~", filedata)
    newdata = timeregex.sub(f"FINAL TIME = {finaltime}\n", tempdata)

    with open(newmodelname,'w') as f:
        f.write(newdata)


def split_voc(vocname, mcsettings, fixparams):
    """Splits .VOC file into multiple versions, for main, country, 
    initial, full model, general MCMC, and country MCMC calibration"""
    with open(vocname,'r') as f0:
        filedata = f0.readlines()
    
    voccty = [line for line in filedata if line[0] == ':' or '[Rgn' in line]

    # Make necessary substitutions for MCMC settings
    vocctymc = ''.join(voccty)
    for k,v in mcsettings.items():
        vocctymc = regex.sub(f":{regex.escape(k)}=.*", f":{k}={v}", vocctymc)
    
    textlist = [voccty, vocctymc]
    sfxs = ['c', 'cmc']
    
    for k, subs in fixparams.items():
        lines = filedata.copy()
        for l, line in enumerate(lines):
            if ':MULTIPLE_START' in line:
                lines[l] = ':MULTIPLE_START=OFF\n'
        text = ''.join(lines)
        for sub, val in subs.items():
            text = regex.sub(f".*{regex.escape(sub)}.*", f"{val}<={sub}<={val}", text)
        textlist.append(text)
        sfxs.append(k)
        
    # Write various voc versions to separate .voc files
    for fname, suffix in zip(textlist, sfxs):
        with open(f"{vocname[:-4]}_{suffix}.voc", 'w') as f:
            f.writelines(fname)


def create_mdls(controlfile, countrylist, finaltime, logfile):
    """Creates copies of the base .mdl file for each country in list 
    (and one main copy) and splits .VOC files"""
    model = controlfile['simsettings']['model']
    for c in countrylist:
        newmodel = model[:-4] + f'_{c}.mdl'
        modify_mdl(c, finaltime, model, newmodel)

    mainmodel = model[:-4] + '_main.mdl'
    c_list = [f'{c}\\\n\t\t' if i % 10 == 9 else c for i,c in enumerate(countrylist)]
    countrylist_str = str(c_list)[1:-1].replace("'","")
    modify_mdl(countrylist_str, finaltime, model, mainmodel)
    split_voc(controlfile['simsettings']['optparm'], 
              controlfile['mcsettings'], controlfile['fixparams'])
    write_log("Files are ready! moving to calibration", logfile)


def read_payoff(outfile, line=1):
    """Identifies payoff value from .OUT or .REP file - 
    use line 1 (default) for .OUT, or use line 0 for .REP"""
    with open(outfile, 'r') as f:
        payoffline = f.readlines()[line]
    payoffvalue = [float(s) for s in 
                   regex.findall(r'-?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?', payoffline)][0]
    return payoffvalue


def read_outvals(outfile):
    """Converts .out file into list of tuples of var names & values"""
    with open(outfile, 'r') as f:
        output = [line for line in f.readlines() if (line[0] != ':')]

    names = [line.split('<=')[1].split('=')[0].strip() for line in output]
    values = [float(line.split('<=')[1].split('=')[1]) for line in output]
    
    return list(zip(names, values))


##### FUNCTION DEFINITIONS FOR ANALYSIS & DATA PROCESSING #####

def get_first_idx(s, threshold):
    """Return index of first value in series `s` above `threshold`"""
    return (s > threshold).idxmax(skipna=True)


def calc_mean(resdf, var, limit=180):
    """Return mean of `var` over historical period given by `limit`"""
    limit = min(len(resdf.loc[var]), limit)
    val = resdf.loc[var][-limit:].mean()
    return val


def calc_gof(resdf, simvar, datavar):
    """Calculate goodness-of-fit measures for given sim & data vars"""
    # IMPORTANT: cross-screen for missing sim or data values
    sim = resdf.loc[simvar].where(resdf.loc[datavar].notna())
    dat = resdf.loc[datavar].where(resdf.loc[simvar].notna())
    
    # Calculate various GOF stats & return each one
    error = abs(sim - dat)
    try:
        maen = error.mean()/dat.mean()
    except ZeroDivisionError:
        maen = np.nan
    mape = (error/dat).mean()
    simstd = np.sqrt((sim ** 2).mean() - sim.mean() ** 2)
    datastd = np.sqrt((dat ** 2).mean() - dat.mean() ** 2)
    r2 = (sim.corr(dat)) ** 2
    mse = (error ** 2).mean()
    um = ((sim.mean() - dat.mean()) ** 2/ mse)
    us = ((simstd - datastd) ** 2/ mse)
    uc = (2 * (1 - sim.corr(dat)) * simstd * datastd / mse)
    return maen, mape, r2, mse, um, us, uc


def trunc_log(df):
    """Return log10 of a dataframe, ignoring negative base values"""
    df[df <= 0] = np.NaN
    return np.log10(df)


def clean_subscripts(varname):
    """Removes 3-letter ISO codes from subscripts in `varname`"""
    name = regex.sub(r"\[[A-Z]{3}]", "", varname)
    name = regex.sub(r"\[[A-Z]{3},\s?", "[", name)
    return name


def process_results(scriptname, gof_vars, c, iqr_list=[], means_list=[], hist_windows=[180], 
                    perc_list=[0.05,0.95], delta=None, duration=None):
    """Read single-country calibration results and calculate additional 
    outputs, incl. percentiles and IQRs, returning a compiled pd.Series 
    of processed country results `c_res` and a Dataframe with country 
    time series outputs (deaths & infs) `datdf`"""
    # Read country parameter values from .out file
    outlist = read_outvals(f'{scriptname}.out')
    varnames = [clean_subscripts(var[0]) for var in outlist]
    vals = [var[1] for var in outlist]
    c_res = pd.Series(vals, index=varnames)
    
    # Read full country calibration results, extract death & inf data for output
    resdf = pd.read_csv(f'{scriptname}.tab', sep='\t', index_col=0, error_bad_lines=False)
    resdf.index = [clean_subscripts(n) for n in resdf.index] # Separate subscripts
    resdf.replace(-999, np.nan, inplace=True)
    datdf = resdf.loc[gof_vars['sim'] + gof_vars['data']]
    
    # Pull end-of-run values from full country results
    endtime = len(resdf.columns) - 1
    c_res['cum_deaths'] = resdf.loc['Dead'][-1]
    c_res['IFR'] = resdf.loc['IFR'][0]
    c_res['SFrac_mdl'] = resdf.loc['Susceptible Fraction'][-1]
    
    # Calculate response-stringency correlation
    try:
        response = resdf.loc['Impact of perceived risk on attack rate']
        response.index = response.index.astype('int64') # Fix index for concatenation
        stringency = table.loc['stringency'].loc[c]
        stringency.index = stringency.index.astype('int64') # Fix index for concatenation

        # Clean variables for regression to prevent PearsonR failure from NaN
        m = pd.concat([response, stringency[:-1]], axis=1).dropna()
        response = response[m.index]
        stringency = stringency[m.index]
        c_res['RS_corr'], c_res['RS_corr_p'] = pearsonr(response, stringency)
    except KeyError:
        c_res['RS_corr'], c_res['RS_corr_p'] = np.nan, np.nan

    # Calculate historical means
    for var in means_list:
        for lim in hist_windows:
            c_res[f"avg_{var}_{lim}"] = calc_mean(resdf, var, limit=lim)
    
    # Calculate GOF statistics
    def gof_by_var(df, gof_vars, sfx=''):
        maeom = []
        r2 = []
        for i, (s, d) in enumerate(zip(gof_vars['sim'], gof_vars['data'])):
            maeom.append(calc_gof(df, s, d)[0])
            r2.append(calc_gof(df, s, d)[2])
            c_res[f'maeom{sfx}_{i}'] = np.mean(maeom[i])
            c_res[f'r2{sfx}_{i}'] = np.mean(r2[i])
        c_res[f'maeom{sfx}'] = np.mean(maeom)
        c_res[f'r2{sfx}'] = np.mean(r2)
    gof_by_var(resdf, gof_vars)
    
    for k in fixparams.keys():
        fixdf = pd.read_csv(f'{scriptname[:-1]}{k}.tab', sep='\t', 
                            index_col=0, error_bad_lines=False)
        fixdf.index = [clean_subscripts(n) for n in resdf.index] # Separate subscripts
        fixdf.replace(-999, np.nan, inplace=True)
        gof_by_var(fixdf, gof_vars, f'_{k}')
        for var in means_list:
            c_res[f"{k}_{var}_{hist_windows[-1]}"] = calc_mean(fixdf, var, limit=hist_windows[-1])
    
    return c_res, datdf


In [ ]:
controlfilename = input("Enter control file name (with extension):")
cf = json.load(open(controlfilename, 'r'))

# Unpack controlfile into variables
for k,v in cf.items():
    exec(k + '=v')

for setting in [datasettings, analysissettings]:
    for k, v in setting.items():
        exec(k + '=v')

# Set up files in run directory and initialise logfile
master = Script(cf)
master.changes.extend(scenariolist)
master.copy_model_files(f"{baserunname}_IterCal")
for f in [f"../{controlfilename}", "../ImportData.cmd", "../CovRegInput.frm"]:
    copy(f, "./")
basedir = os.getcwd()
basename = cf['baserunname']
logfile = f"{basedir}/{baserunname}.log"
write_log(f"-----\nStarting new log at {time.ctime()}\nReady to work!", logfile)

### Data updating

In [ ]:
##### RAW DATA INTAKE & PRE-PROCESSING #####

# Calculate cutoff dates
startdate = datetime.date.fromisoformat('2019-12-31')
enddate = startdate + datetime.timedelta(days=end_date+1)
em_enddate = startdate + datetime.timedelta(days=end_date//7 * 7 - 1) # Excess mortality data only reported weekly

if updatedata != 0:
    # Read main, mobility, and excess mortality data from URL for raw data CSVs
    dat_raw = pd.read_csv(data_url)
    mobdata = pd.read_csv(mobdata_url)
    em_data = pd.read_csv(emdata_url)

    # Extract dictionary mapping of ISO codes to OWID country names
    names = dat_raw.filter(['iso_code','location'], axis=1).drop_duplicates()
    names.replace({'iso_code': renames}, inplace=True) # Rename unusual ISO codes as needed
    c_dict = dict(zip(names['location'], names['iso_code']))

    # Extract vaccination-relevant data for later use
    dat_em = dat_raw.filter(['iso_code', 'date', 'total_deaths', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred'], axis=1)

    # Extract excess mortality data from The Economist
    em_data = em_data.filter(['iso3c', 'date', 'cumulative_estimated_daily_excess_deaths'], axis=1)
    em_data.columns = ['iso_code', 'date', 'cumulative_excess_mortality']

    # Subset vaccination & excess mortality data to specific dates needed
    dat_em = dat_em.loc[dat_em['date']==str(enddate)].drop('date', axis=1)
    em_data = em_data.loc[em_data['date']==str(em_enddate)].drop('date', axis=1)

    # Subset main CSV to relevant data fields
    data = dat_raw.filter(['iso_code', 'date', 'total_cases', 'new_cases_smoothed', 
                        'new_deaths_smoothed', 'reproduction_rate', 
                        'stringency_index', 'population', 'gdp_per_capita', 
                        'hospital_beds_per_thousand', 'median_age'], axis=1)

    # Rename fields as needed
    data.columns = ['iso_code','date', 'total_cases_owid', 'new_cases_owid', 'new_deaths_owid', 'Re_est', 
                    'stringency', 'population', 'gdp_per_capita', 'hosp_beds', 'median_age']

    # Read and merge IHME data files
    ihme0, ihme1, ihme2, ihme3 = [pd.read_csv(f"{ihme_url_partial}_{year}.csv") for year in [2020, 2021, 2022, 2023]]

    # Subset CSV to relevant data fields
    ihme = [i.filter(['date', 'location_name', 'inf_cuml_mean', 'inf_mean', 'daily_deaths'], axis=1) 
            for i in [ihme0, ihme1, ihme2, ihme3]]
    del ihme0, ihme1, ihme2, ihme3

    ihme = pd.concat(ihme)

    # Rename fields as needed
    ihme.columns = ['date', 'iso_code', 'total_cases_ihme', 'new_cases_ihme', 'new_deaths_ihme']
    ihme.replace({'iso_code': c_dict}, inplace=True) # Convert country names to ISO codes
    ihme.replace({'iso_code': ihme_c_dict}, inplace=True)

    # Merge OWID and IHME data
    data = data.merge(ihme, how='left', on=['iso_code', 'date'])

    # Designate specified dataset case & death columns for main analysis
    if dataset == 'OWID':
        data['total_cases'] = data['total_cases_owid']
        data['new_cases'] = data['new_cases_owid']
        data['new_deaths'] = data['new_deaths_owid']
    elif dataset == 'IHME':
        data['total_cases'] = data['total_cases_ihme']
        data['new_cases'] = data['new_cases_ihme']
        data['new_deaths'] = data['new_deaths_ihme']

    table = pd.pivot_table(
        data, values=['total_cases', 'new_cases', 'new_deaths', 'Re_est', 'stringency', 'population', 
                      'gdp_per_capita', 'hosp_beds', 'median_age'], index='date', columns='iso_code')
    table = table.T
    table.index.names = ['field', 'iso_code']
    table.columns = pd.to_datetime(table.columns)

    # Save next 6 mo of data for forecast comparison
    tab_forecast = table.iloc[:, end_date+1:(end_date + max(hist_windows) + 1)]
    table = table.iloc[:, :end_date+1] # Then cut off table at end date

    # Drop countries with fewer cases than specified threshold, insufficient datapoints, or zero deaths
    dropidx_cases = table.loc['total_cases'].index[table.loc['total_cases'].max(axis=1) < min_cases]
    dropidx_deaths = table.loc['new_deaths'].index[table.loc['new_deaths'].max(axis=1) == 0]
    first_idxs = (table.loc['total_cases'] > start_cases).idxmax(axis=1)
    dropidx_data = table.loc['total_cases'].index[
        (table.columns[-1] - first_idxs).dt.days < min_datapoints]
    print("Insufficient cases: ", dropidx_cases, "\nInsufficient deaths: ", dropidx_deaths, 
          "\nInsufficient datapoints: ", dropidx_data, "\nMisc. removals: ", droplist)
    for drops in [dropidx_cases, dropidx_deaths, dropidx_data, droplist]:
        table.drop(drops, level='iso_code', inplace=True, errors='ignore')
        tab_forecast.drop(drops, level='iso_code', inplace=True, errors='ignore')

    table = table.rename(index=renames) # Rename any unusual ISO codes as needed
    tab_forecast = tab_forecast.rename(index=renames)

    # Convert column indices to day number since startdate
    table.columns = (table.columns - pd.to_datetime('2019-12-31')).days
    tab_forecast.columns = (tab_forecast.columns - pd.to_datetime('2019-12-31')).days

    # Reorder multiindex levels before by-country subsetting
    table = table.reorder_levels(['iso_code', 'field']).sort_index()

    # Identify first date over infection threshold for each country and subset dataframe accordingly
    dropidx_errors = []
    r0dict = {}
    for i in table.index.levels[0]:
        try:
            first_idx = get_first_idx(table.loc[i].loc['total_cases'], start_cases)
            table.loc[i].loc[:, :first_idx] = np.NaN
            r0dict[i] = table.loc[i].loc['Re_est', max( # Get first point Re value (+1 to avoid NaN)
                first_idx+1, table.loc[i].loc['Re_est'].first_valid_index())]
        except KeyError:
            table.drop(i, level='iso_code', inplace=True, errors='ignore')
            dropidx_errors.append(i)
    print("Other data errors: ", dropidx_errors)
    r0 = pd.Series(r0dict) # Create series of first point Re values (initial RR, i.e. R0)

    # Clean infinite values and switch multiindex levels back
    table.replace([np.inf, -np.inf], np.NaN, inplace=True)
    table = table.reorder_levels(['field', 'iso_code']).sort_index()

    # If applicable, drop early data for sensitivity analysis
    if early_data_cutoff != 0:
        table = table.loc[:, early_data_cutoff+1:]

    # Move country statistics columns to separate dataframe
    statdict = {'R0_est': r0}
    for stat in ['population', 'gdp_per_capita', 'hosp_beds', 'median_age']:
        statdict[stat] = table.loc[stat].mean(axis=1)
        table.drop(stat, level='field', inplace=True, errors='ignore')

    cstats = pd.concat(statdict, axis=1)

    # Calculate aggregate data over last `hist_window` days and add to country stats dataframe
    for lim in hist_windows:
        cstats[f'hist_deaths_{lim}'] = table.loc['new_deaths'].iloc[:, -lim:].mean(axis=1)
        cstats[f'hist_Re_{lim}'] = table.loc['Re_est'].iloc[:, -lim:].mean(axis=1)
        cstats[f'hist_str_{lim}'] = table.loc['stringency'].iloc[:, -lim:].mean(axis=1)
        cstats[f'hist_dpm_{lim}'] = cstats[f'hist_deaths_{lim}'] / cstats['population'] * 1e06

        # As well as aggregate data from forecast window
        cstats[f'next_deaths_{lim}'] = tab_forecast.loc['new_deaths'].iloc[:, :lim].mean(axis=1)
        cstats[f'next_Re_{lim}'] = tab_forecast.loc['Re_est'].iloc[:, :lim].mean(axis=1)
        cstats[f'next_str_{lim}'] = tab_forecast.loc['stringency'].iloc[:, :lim].mean(axis=1)
        cstats[f'next_dpm_{lim}'] = cstats[f'next_deaths_{lim}'] / cstats['population'] * 1e06

    # Extract mobility change data to pivot table
    mobdata['average'] = pd.concat([mobdata['retail_and_recreation'], mobdata['workplaces']], 
                                   axis=1).mean(axis=1) # Get average of R&R and workplace values
    mobdata.replace({'Country': c_dict}, inplace=True) # Convert country names to ISO codes
    mobtable = pd.pivot_table(mobdata, values=['retail_and_recreation', 'workplaces', 'average'], 
                              index='Year', columns='Country')
    mobtable = mobtable.T

    # Calculate averages over last `hist_window` days & recompile into new dataframe
    mobend = end_date + 1 - mobtable.columns[0] # Mobility data starts 17 Feb 2020
    mobstart = mobend - hist_windows[-1]
    mobtable = mobtable[mobtable.columns[mobstart:mobend]]
    tbm = mobtable.mean(axis=1)
    mobmean = pd.concat([tbm.loc['average'], tbm.loc['retail_and_recreation'], tbm.loc['workplaces']], 
                        keys=['mob_avg', 'mob_rr', 'mob_wk'], axis=1)

    cstats = cstats.merge(mobmean, how='left', left_index=True, right_index=True)

    # Extract countries over vaccination threshold to save separately
    data_vax = pd.concat([dat_em.loc[dat_em['iso_code']=='OWID_WRL'], 
                          (dat_em.loc[dat_em['iso_code'].isin(table.index.get_level_values(1))]
                           .loc[dat_em['people_vaccinated_per_hundred'] >= vax_threshold])])

    # Combine excess mortality data and calculate EM threshold flags
    dat_em = dat_em.merge(em_data, how='left', on='iso_code')
    dat_em['excess_mortality_fraction'] = dat_em['cumulative_excess_mortality'] / dat_em['total_deaths']
    dat_em['em_100'] = dat_em['excess_mortality_fraction'] < 2 # Indicates EM < 100% more than reported deaths
    dat_em['em_50'] = dat_em['excess_mortality_fraction'] < 1.5
    dat_em['em_25'] = dat_em['excess_mortality_fraction'] < 1.25

    cstats = cstats.merge(dat_em.set_index('iso_code'), how='left', left_index=True, right_index=True)

    # Export processed dataframes to .tab and import to VDF
    table.to_csv('./InputData.tab', sep='\t')
    subprocess.run(f"{vensim7path} \"./ImportData.cmd\"", check=True)
    data_vax.to_csv('./VaxData.tab', sep='\t')
    cstats.to_csv('./CountryStats.tab', sep='\t')

else: # Or read in existing .tab
    table = pd.read_csv(f'./InputData.tab', sep='\t', index_col=[0,1])
    cstats = pd.read_csv('./CountryStats.tab', sep='\t', index_col=0)

display(table)
display(cstats)
    
# Update FinalTime cin with last day of available data - IMPORTANT! USES FIRST FILE IN CHANGES LIST
finaltime = len(table.columns)-1
with open(simsettings['changes'][0], 'w') as f:
    f.write(f"FINAL TIME = {finaltime}")

### Main model calibration

In [ ]:
##### MAIN ANALYSIS, DURATION SENSITIVITY & RESULTS-PROCESSING CODE #####

os.chdir(basedir)

# Pull country list from data table
table.index = table.index.remove_unused_levels()
countrylist = list(table.index.levels[1])
# countrylist = ['FRA', 'GBR', 'USA', 'ZAF']

print(countrylist)

# Loop through disease duration values to test, starting with main then sensitivity values
for i in ([main_dur] + sens_durs):
    cf['baserunname'] = f'{basename}{i}'
    baserunname = cf['baserunname']
    print(baserunname)

    # Create script object for given duration, to cleanly create calibration subfolder
    sub = Script(cf)
    sub.changes.extend(scenariolist)
    sub.copy_model_files(baserunname)
    copy(f"../{controlfilename}", "./")
    
    # Overwrite disease duration cin file - IMPORTANT! USES LAST FILE IN CHANGES LIST
    with open(simsettings['changes'][-1], 'w') as f:
        f.write(f"DiseaseDuration = {i}")
        
    dur = i # Assign disease duration variable
    
    # Initialise necessary .mdl and .voc files
    create_mdls(cf, countrylist, finaltime, logfile)
    
    # Run country-by-country calibration process, unless otherwise specified (mccores=0)
    if mccores != 0:
        write_log(f"Initialising MCMC with duration {dur}!", logfile)
        c_list = []
        err_list = []

        for c in countrylist:
            while True:
                try:
                    res_i = compile_script(cf, CtyScript, c, 'i', {'model': f'_{c}', 'optparm': '_c'}, 
                                           logfile, subdir=c)
                    if res_i != False:
                        res_nbr = compile_script(
                            cf, CtyScript, c, 'nbr', {'model': f'_{c}', 'optparm': '_nbr'}, 
                            logfile, subdir=c)
                        c_list.append(c) # Compile updated c_list of successful calibrations
                    else:
                        err_list.append(c) # Compile error list of failed calibrations
                    time.sleep(1)
                    break
                except FileNotFoundError:
                    os.chdir(f"{basedir}/{baserunname}")
                    continue
                
        write_log(f"Calibration complete! Error list is:\n{err_list}", logfile)
    
    # If calibration not needed, default to using country list from data as c_list
    else:
        write_log("Hang on to outdated imperialist dogma! Using previous output...", logfile)
        c_list = countrylist
        err_list = []
    
    os.chdir("..") # Remember to go back to root directory before next iteration!

##### PROCESS CALIBRATION RESULTS #####
write_log("Processing results!", logfile)

# Initialise containers for processed country results and death data
res_list = []
dat_list = []

# Loop through country MCMC outputs, calling master results processing function on each
for c in c_list:
    try:
        print(f"Processing {c}!")
        c_res, datdf = process_results(f'./{c}/{baserunname}_{c}_i', gof_vars, c, iqr_list, 
                                       means_list, hist_windows, perc_list, delta, dur)
        print(f"{c} successful!")
        res_list.append(c_res)
        dat_list.append(datdf)
    except FileNotFoundError:
        err_list.append(c)

display(err_list)

# Compile main results dataframe with processed country results
results = pd.concat(res_list, axis=1)

# Compile country infection and death outputs over time
gof_results = [
    pd.concat([df.loc[var] for df in dat_list], axis=1) for var 
    in gof_vars['sim'] + gof_vars['data']]

# Assign results dataframe indices based on c_list
for df in [results] + gof_results:
    df.columns = [c for c in c_list if c not in err_list]
results = results.T
gof_results = [df.T for df in gof_results]

# Recompile results dataframe with aggregate data previously separated
results = results.merge(cstats, how='left', left_index=True, right_index=True)

# Calculate additional results
results['alp[Death]'] = results['alp[Infection]'] * results['alpratio']
#     results['Alpha K'] = results['Alpha 0'] * (results['Alpha Rel'] - 1) / end_date
for lim in hist_windows:
    results[f"avg_dpm_{lim}"] = results[f"avg_Outputs[Death]_{lim}"] / results['population'] * 1e06

results['hist_dpm_norm'] = results['hist_dpm_180'] / results['IFR'] * results['IFR'].mean()
results['avg_dpm_norm'] = results['avg_dpm_180'] / results['IFR'] * results['IFR'].mean()
results['next_dpm_norm'] = results['next_dpm_180'] / results['IFR'] * results['IFR'].mean()
results['g_D_180'] = 1 / (1 + results['avg_Sensitivity Alpha_180'] * results['avg_dpm_180'])
results['marginal'] = 1 / (1 + results['avg_Sensitivity Alpha_180'])
results['half_contact_dpm'] = 1/ results['avg_Sensitivity Alpha_180']

display(results)

gofdf = pd.concat(gof_results, keys=gof_vars['sim']+gof_vars['data'])
gofdf.index.names=['field', 'iso_code']
display(gofdf)


results.to_csv(f'./{baserunname}_results.tab', sep='\t')
gofdf.to_csv(f'./{baserunname}_fits.tab', sep='\t')
copy(f'./{baserunname}_results.tab', '../')
copy(f'./{baserunname}_fits.tab', '../')

os.chdir("..") # Remember to go back to root directory before next iteration!
    

### Graphing & further results processing

In [ ]:
os.chdir(basedir)
figext = 'png'
r = pd.read_csv(f'./{baserunname}_results.tab', sep='\t', index_col=0)
# Exclude outliers where responsiveness failed to estimate accurately
r = r[r['avg_Sensitivity Alpha_180'] > 0.002]

In [ ]:
### Paper Exhibit 1 & Figures S5, S6 (forecast deaths vs. responsiveness scatter) ###

xvar = 'avg_Sensitivity Alpha_180'
yvar = 'next_dpm_180'
threshold = 5e07

fig0, ax0 = plt.subplots(figsize=[7.5, 7], constrained_layout=True)

# Format X and Y axes with log scales if specified
ax0.set_xscale('log')
ax0.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
ax0.set_yscale('log')
ax0.yaxis.set_major_formatter(mticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

# Plot basic scatterplot of all countries
ax0.scatter(r[xvar], r[yvar], c='gray')

# Plot highlight scatterplot of countries with population above `threshold`
p = r[r['population'] > threshold]
p = p[p['em_100'] == True]
ax0.scatter(p[xvar], p[yvar])
for i in p.index: # Label points with country abbreviations
    ax0.annotate(f' {i}', (p[xvar][i], p[yvar][i]), fontsize=8)

# Take log of variables for regression if specified
x = trunc_log(r[xvar])
y = trunc_log(r[yvar])

# Clean variables for regression to prevent PearsonR failure from NaN
m = pd.concat([x, y], axis=1).dropna()
x = x[m.index]
y = y[m.index]

# Fit basic OLS model
mod_OLS = sm.OLS(y, sm.add_constant(x), missing='drop')
fit_reg = mod_OLS.fit()
display(fit_reg.params)
display(fit_reg.pvalues)

# Plot best-fit regression line, log-scaled if necessary
# ax0.plot(r[xvar][m.index], np.exp(fit_reg.fittedvalues), c='r', ls='dotted')

# Label X and Y axis
ax0.set_xlabel(r'Responsiveness $\alpha$', fontsize=12)
ax0.set_ylabel('Deaths per million (next 6 mo avg)', fontsize=12)

# Set up and annotate with summary stats
# stats = (f"Slope = {'{0:.3g}'.format(fit_reg.params[1])}\n"
#          r"$R^2$" f" = {'{0:.3g}'.format(fit_reg.rsquared)}")
os_corr = '{0:.3g}'.format(x.corr(y))
stats = (f"Correlation = {os_corr}")
ax0.text(0.05, 0.05, stats, transform=ax0.transAxes, backgroundcolor='white', fontsize=14, horizontalalignment='left')

fig0.savefig(f"./{baserunname}_dpm_alpha_next_noline.{figext}", bbox_inches='tight')
# fig0.savefig(f"./{baserunname}_dpm_alpha_next.{figext}", bbox_inches='tight')

alpha_r2 = (xvar, yvar, fit_reg.rsquared)

# Save figure data to Excel for plot
r[[xvar, yvar]].to_excel(f'./{baserunname}_excelfig.xlsx')

In [ ]:
### Paper Figure S3 (additional potential explanatory variables) ###

def plot_partial(r, ax, xvar, yvar, xlog=False, ylog=False, xticks=False, x_label='', y_label='', threshold=0):
    # Format X and Y axes with log scales if specified
    if xlog == True:
        ax.set_xscale('log')
        ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))
    if xticks == True:
        ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: '{:g}%'.format(x * 100)))
        ax.xaxis.set_minor_formatter(mticker.FuncFormatter(lambda x, _: '{:g}%'.format(x * 100)))
        ax.tick_params(axis='x', which='both', labelsize=8)
    if ylog == True:
        ax.set_yscale('log')
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

    # Plot basic scatterplot of all countries
    ax.scatter(r[xvar], r[yvar], c='gray', alpha=0.5)

    # Plot highlight scatterplot of countries with population above `threshold`
    p = r[r['population'] > threshold]
    ax.scatter(p[xvar], p[yvar])
    for i in p.index: # Label points with country abbreviations
        ax.annotate(f' {i}', (p[xvar][i], p[yvar][i]), fontsize=8)

    # Take log of variables for regression if specified
    if xlog == True:
        x = trunc_log(r[xvar])
    else: x = r[xvar]
    if ylog == True:
        y = trunc_log(r[yvar])
    else: y = r[yvar]

    # Clean variables for regression to prevent PearsonR failure from NaN
    m = pd.concat([x, y], axis=1).dropna()
    x = x[m.index]
    y = y[m.index]

    # Fit basic OLS model
    mod_OLS = sm.OLS(y, sm.add_constant(x), missing='drop')
    fit_reg = mod_OLS.fit()
    display(fit_reg.params)

    # Plot best-fit regression line, log-scaled if necessary
    if ylog == True:
        ax.plot(r[xvar][m.index], np.exp(fit_reg.fittedvalues), c='r', ls='dotted')
    else:
        ax.plot(r[xvar][m.index], fit_reg.fittedvalues, c='r', ls='dotted')
        
    # Label X and Y axis
    ax.set_xlabel(x_label, fontsize=12)
    ax.set_ylabel(y_label, fontsize=12)

    # Set up and annotate with summary stats
    stats = (f"Slope = {'{0:.3g}'.format(fit_reg.params[1])}\n"
             r"$R^2$" f" = {'{0:.3g}'.format(fit_reg.rsquared)}")
    ax.text(0.95, 0.05, stats, transform=ax.transAxes, backgroundcolor='white', fontsize=12, ha='right')
    
    return (xvar, yvar, fit_reg.rsquared)


# Set up basic figure and axes
fig0, ((ax0, ax1, ax2), (ax3, ax4, ax5)) = plt.subplots(2, 3, figsize=[18, 12], constrained_layout=True, sharey=True)

alp_r2 = plot_partial(r, ax0, 'avg_Sensitivity Alpha_180', 'next_dpm_180', True, True, False, 
                      r'Responsiveness $\alpha$', 'Deaths per million (prediction period avg)', threshold=5e07)
IFR_r2 = plot_partial(r, ax1, 'IFR', 'next_dpm_180', True, True, True,  
                      'Infection fatality rate (%)', threshold=5e07)
GDP_r2 = plot_partial(r, ax2, 'gdp_per_capita', 'next_dpm_180', True, True, False,  
                      'GDP per capita', threshold=5e07)
hosp_r2 = plot_partial(r, ax3, 'hosp_beds', 'next_dpm_180', False, True, False, 
                       'Hospital beds per thousand people', 'Deaths per million (prediction period avg)', threshold=5e07)
R0_r2 = plot_partial(r, ax4, 'R0_est', 'next_dpm_180', False, True, False, 
                     r'Initial reproduction number $R_0$', threshold=5e07)
str_r2 = plot_partial(r, ax5, 'next_str_180', 'next_dpm_180', False, True, False, 
                      'OxCGRT stringency index (6 month average)', threshold=5e07)

for ax, letter in zip([ax0, ax1, ax2, ax3, ax4, ax5], ['A', 'B', 'C', 'D', 'E', 'F']):
    ax.text(0.03, 0.97, letter, transform=ax.transAxes, backgroundcolor='white', 
            fontsize=12, weight='bold', va='top')

fig0.savefig(f"./{baserunname}_dpm_corrs_exp.{figext}", bbox_inches='tight')

In [ ]:
### Paper Figure S4 (mobility scatterplot) ###
fig0, (ax0, ax1) = plt.subplots(1, 2, figsize=[12, 6], constrained_layout=True, sharey=True)

mobrr_r2 = plot_partial(r, ax0, 'mob_rr', 'avg_dpm_180', False, True, False, 
                        'Change in mobility, retail & recreation (6 mo avg)', 'Deaths per million (6 mo avg, estimated)', threshold=5e07)
mobwk_r2 = plot_partial(r, ax1, 'mob_wk', 'avg_dpm_180', False, True, False, 
                        'Change in mobility, workplaces (6 mo avg)', threshold=5e07)

for ax, letter in zip([ax0, ax1], ['A', 'B']):
    ax.text(0.03, 0.97, letter, transform=ax.transAxes, backgroundcolor='white', 
            fontsize=12, weight='bold', va='top')

fig0.savefig(f"./{baserunname}_mobility_supp.{figext}", bbox_inches='tight')

#### Regression analyses

In [ ]:
### Paper Exhibit 2 (main mortality regression) & Tables S6-S12 (robustness checks) ###

params = ['Beta', 'Alpha 0', 'Time to increase risk', 'Time to reduce risk', 
          'Patient Zero Arrival Time', 'alp[Infection]', 'alp[Death]']

paramdict = {}
for p in params:
    paramdict[p] = {'Mean': '{0:.3g}'.format(r[p].mean()), 
                    'Median': '{0:.3g}'.format(r[p].median()), 
                    'StDev': '{0:.3g}'.format(r[p].std())}
paramdf = pd.DataFrame(paramdict).T
paramdf.to_csv(f'./{baserunname}_params.tab', sep='\t')

pd.set_option('mode.chained_assignment', None) # Suppress warnings

for frac in ['all', '100', '50', '25', 'vax_part', 'vax_full']:
    # Define predictor variables to include in regression, log-transformed or not
    reg_list = ['hosp_beds', 'R0_est', 'next_str_180']
    reg_log_list = ['avg_Sensitivity Alpha_180', 'IFR', 'gdp_per_capita']

    # Drop countries with missing data & subset by excess mortality fraction
    if frac == 'all':
        t = r.dropna(subset=reg_list + reg_log_list + ['next_dpm_180'])
    elif frac == 'vax_full':
        t = (r[(r['people_fully_vaccinated_per_hundred'] < vax_threshold)
               |(pd.isnull(r['people_fully_vaccinated_per_hundred']))]
             .loc[r[f'em_100']==True]
             .dropna(subset=reg_list + reg_log_list + ['next_dpm_180']))
    elif frac == 'vax_part':
        t = (r[(r['people_vaccinated_per_hundred'] < vax_threshold)
               |(pd.isnull(r['people_vaccinated_per_hundred']))]
             .loc[r[f'em_100']==True]
             .dropna(subset=reg_list + reg_log_list + ['next_dpm_180']))
    else:
        t = r.loc[r[f'em_{frac}']==True].dropna(subset=reg_list + reg_log_list + ['next_dpm_180'])

    # Take log for outcome and specified predictors
    for var in reg_log_list + ['next_dpm_180']:
        t.loc[:, f'log_{var}'] = trunc_log(t[var])

    # Compile list of predictor variables
    varlist = [f'log_{var}' for var in reg_log_list] + reg_list

    # Drop countries with errors from log transform
    t.dropna(subset=varlist + ['log_next_dpm_180'], inplace=True)

    # Specify predictors & outcome for more compact code
    y = t['log_next_dpm_180']
    x = t[varlist]

    # Define whether to add constant or not
    reg_const = 0

    # Create and fit OLS, including intercept if specified
    if reg_const:
        mod_OLS_f = sm.OLS(y, sm.add_constant(x), missing='drop')
    else:
        mod_OLS_f = sm.OLS(y, x, missing='drop')
    fit_reg_f = mod_OLS_f.fit()
    display(fit_reg_f.summary())

    # Save full results
    with open(f'{baserunname}_reg_raw_{frac}.csv', 'w') as f:
        f.write(fit_reg_f.summary().as_csv())

    # Record R2 and adjusted R2 values
    r2_f, r2a_f = fit_reg_f.rsquared, fit_reg_f.rsquared_adj

    # Initialise containers for results
    r2dict, r2adict, effdict, efldict, efhdict = [pd.Series() for i in range(5)]

    # Drop each individual predictors and re-fit model to calculate contributions
    for var in varlist:
        sublist = [v for v in varlist if v != var]
        x = t[sublist]

        if reg_const:
            mod_OLS = sm.OLS(y, sm.add_constant(x), missing='drop')
        else:
            mod_OLS = sm.OLS(y, x, missing='drop')
        fit_reg = mod_OLS.fit()

        r2dict[var] = r2_f - fit_reg.rsquared
        r2adict[var] = r2a_f - fit_reg.rsquared_adj

        # Calculate effect sizes and bounds
        effdict[var] = 10 ** (fit_reg_f.params[var] * (t[var].std()))
        efldict[var] = 10 ** ((fit_reg_f.params[var] - 1.96 * fit_reg_f.HC0_se[var]) * (t[var].std()))
        efhdict[var] = 10 ** ((fit_reg_f.params[var] + 1.96 * fit_reg_f.HC0_se[var]) * (t[var].std()))

    # Define and concatenate relevant results variables
    r_c, r_se, r_t, r_p = fit_reg_f.params, fit_reg_f.HC0_se, fit_reg_f.tvalues, fit_reg_f.pvalues
    reg_res = pd.concat([r_c, r_se, r_t, r_p, r2dict, r2adict, effdict, efldict, efhdict], keys=[
        'Coeff', 'Std Err', 't', 'P>|t|', r'$R^2$', r'Adj. $R^2$', 'Eff. size', 'Eff. 2.5%', 'Eff 97.5%'], axis=1)

    # Save regression results & predictor correlation matrix
    reg_res.to_csv(f'./{baserunname}_regression_{frac}.tab', sep='\t')
    t[varlist].corr().to_csv(f'./{baserunname}_regcorrs_{frac}.tab', sep='\t')
    display(reg_res)
    display(t[varlist].corr())

    # Calculate and save variance inflation factors
    xdf = sm.add_constant(t[varlist])
    vifs = pd.Series([variance_inflation_factor(xdf.values, i) 
                      for i in range(xdf.shape[1])], index=xdf.columns)
    vifs.drop('const', inplace=True)
    vifs.to_csv(f'./{baserunname}_vifs_{frac}.tab', sep='\t')
    display(vifs)

pd.set_option('mode.chained_assignment', 'warn') # Unsuppress warnings

In [ ]:
##### CREATE KEY RESULTS SUMMARY FILE #####
with open(cf['simsettings']['optparm']) as f:
    restartline = [line for line in f.readlines() if ':RESTART_MAX' in line]
    restarts = int(restartline[0].split('=')[1])

# Compile summarytext
summarytext = [
    f"Total countries\t{len(r.index)}\tPopulation\t{int(r['population'].sum())}\n", 
    f"Dates\t{startdate.isoformat()}\t{enddate.isoformat()}\tDays\t{end_date}\n", 
    f"Min cumulative cases\t{min_cases}\tMin datapoints\t{min_datapoints}\tStartpoint cases\t{start_cases}\n", 
    f"Restarts\t{restarts}\n", 
    f"Hist DPM\t{r['hist_dpm_180'].quantile(0.05)}\t{r['hist_dpm_180'].median()}\t{r['hist_dpm_180'].quantile(0.95)}\n", 
    f"IFR\t{r['IFR'].quantile(0.05)}\t{r['IFR'].median()}\t{r['IFR'].quantile(0.95)}\n", 
    f"R0_est\t{r['R0_est'].quantile(0.05)}\t{r['R0_est'].median()}\t{r['R0_est'].quantile(0.95)}\n", 
    f"Alpha\t{r['avg_Sensitivity Alpha_180'].quantile(0.05)}\t{r['avg_Sensitivity Alpha_180'].median()}\t{r['avg_Sensitivity Alpha_180'].quantile(0.95)}\n", 
    f"In-sample stats\tCorrelation\t{is_corr}\tSlope\t{is_slope}\tR2\t{is_r2}\n", 
    f"Out-of-sample stats\tCorrelation\t{os_corr}\n", 
    f"Response-stringency correlation\tMean\t{r['RS_corr'].mean()}\tMedian\t{r['RS_corr'].median()}\n", 
    f"Excess mortality thresholds\tEM100\t{sum(r['em_100'])}\tEM50\t{sum(r['em_50'])}\tEM25\t{sum(r['em_25'])}\n", 
    f"Countries w/ >10% vax\tPartial vax\t{len(r.loc[r['people_vaccinated_per_hundred'] >= 10])}\tFull vax\t{len(r.loc[r['people_fully_vaccinated_per_hundred'] >= 10])}\n"
]
    
# Export summary text file
with open(f"{baserunname}_summary.tab", 'w') as summaryfile:
    summaryfile.writelines(summarytext)

---
### End of main analysis
---

In [ ]:
000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000